In [ ]:
###########import packages##########
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from tensorflow.keras.backend.tensorflow_backend import set_session
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
#from tensorflow.keras.constraints import maxnorm 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
%matplotlib

###########fix random seed for reproducability##########
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

'''
gpu_options = tf.GPUOptions(allow_growth=True)
gpu_options =tf.GPUOptions(per_process_gpu_memory_fraction=0.8,allow_growth=True) ##每个gpu占用0.8存
config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True)
sess=tf.Session(config=config)
'''
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
csv = "../../database/database422.csv"
file = open('../../results/ANN_Opt_parm/parameter.txt','w')

fdata=pd.read_csv(csv,encoding="gbk", index_col=0)
raw_data = fdata.iloc[:, :]
###########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)#即简单实现标准化
###########train test splitting##########
param=standardized_data.iloc[:,0:422]
power=standardized_data.iloc[:,422]
X=param.values.astype(np.float32)
y=power.values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, random_state=751)

###########search neuron network hyperparmeter space##########
neurons=750
max_corr_ann=0
for activation in ['relu','tanh','sigmoid','softsign']:
    for regularizer_term in [0.0001,0.0005,0.001,0.005,0.01,0.05,0.1]:
        for dropout in [0.1,0.5]:
            for epochs_number in range(1000,1200):
                for batch_size_number in range (20,200,40):
                    for learning_rate_search in [0.001,0.005,0.01,0.05]:
                        ###########implementing hyperparameters##########
                        neurons1=neurons
                        activation1=activation
                        regularizer=keras.regularizers.l2(regularizer_term)
                        dropout_rate=dropout
                        ###########keras ANN model construction##########
                        model = Sequential() 
                        model.add(Dense(neurons1, input_dim=422, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                    bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate)) 
                        model.add(Dense(1, input_dim=neurons1, activation='linear'))
                        adam=optimizers.Adam(lr=learning_rate_search)
                        model.compile(loss='mse', optimizer=adam) 
                        #print('Training ------------')
                        ###########train the model with the training set##########
                        ###########testset has been remove before##########
                        model.fit(X_train, y_train,verbose=0, epochs=epochs_number, batch_size=batch_size_number,validation_split=0.1,callbacks=[TensorBoard(log_dir='mytensorboard')])
                        loss= model.evaluate(X_test, y_test)
                        predict_ann= model.predict(X_test)
                        train_ann= model.predict(X_train)
                        ###########result output##########
                        x_prediction_maximum_power_ann=predict_ann*np.std(data,axis=0)[422]+np.mean(data,axis=0)[422]
                        y_real_maximum_power=y_test*np.std(data,axis=0)[422]+np.mean(data,axis=0)[422]
                        x_prediction_maximum_power_ann=x_prediction_maximum_power_ann[:,0]
                        x_prediction_maximum_power_ann_series=pd.Series(x_prediction_maximum_power_ann)
                        y_real_maximum_power_series=pd.Series(y_real_maximum_power)
                        corr_ann = round(x_prediction_maximum_power_ann_series.corr(y_real_maximum_power_series), 4)
                        rmse_val= rmse(x_prediction_maximum_power_ann,y_real_maximum_power)
                        print('ANN,R2',corr_ann,'RMSE',rmse_val)
                        if corr_ann>0.85:
                            mm1 ='%f'%corr_ann
                            mw1 ='%f'%rmse_val
                            neurons1 = '%d'%neurons
                            regularizer_term1='%f'%regularizer_term
                            dropout1='%f'%dropout
                            epochs_number1='%f'%epochs_number
                            batch_size_number1='%f'%batch_size_number
                            file.write('ANN,R2:'+' '+ mm1 +' '+ 'RMSE:'+' '+mw1+ '\n')
                            file.write('neurons:'+' '+ neurons1 +' '+ 'regularizer_term:'+' '+regularizer_term1+ '\n')
                            file.write('dropout:'+' '+ dropout1 +' '+ 'batch_size_number:'+' '+batch_size_number1+ '\n')
                            if corr_ann > max_corr_ann:
                                max_corr_ann = corr_ann
                                max_rmse_val = rmse_val
                                max_y_real_maximum_power = y_real_maximum_power
                                max_x_prediction_maximum_power_ann = x_prediction_maximum_power_ann
                                max_learning_rate_search = learning_rate_search
                                max_neurons = neurons
                                max_activation = activation
                                max_regularizer_term = regularizer_term
                                max_dropout = dropout
                                max_epochs_number = epochs_number
                                max_batch_size_number = batch_size_number
                        if 0.9<corr_ann<1:
                            break
                        else:
                            K.clear_session()
                    else:continue
                    break
                else:continue
                break
            else:continue
            break
        else:continue
        break
    else:continue
    break

###########print best hyperparameter##########
print(learning_rate_search)
print(neurons)
print(activation)
print(regularizer_term)
print(dropout)
print(epochs_number)
print(batch_size_number)
print('ANN,R2',corr_ann)
print('ANN,RMSE',rmse_val)
print(y_real_maximum_power)
###########visualization##########
x_y_x=np.arange(0,100,10)
x_y_y=np.arange(0,100,10)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(x_prediction_maximum_power_ann,y_real_maximum_power,color='red',label='Artificial Neural Network')
plt.legend()
ax.plot(x_y_x,x_y_y)
plt.xlabel(u"Predicted_Degradable_Conductivity %")
plt.ylabel(u"Real_Degradable_Conductivity %")
plt.show()

In [ ]:
model.save('ANN.h5')